<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data"

cols = ["Class Name", "handicapped-infants", "water-project-cost-sharing",
        "adoption-of-the-budget-resolution", "physician-fee-freeze", 
        "el-salvador-aid", "religious-groups-in-schools",
        "anti-satellite-test-ban", "aid-to-nicaraguan-contras", "mx-missile",
        "immigration", "synfuels-corporation-cutback", "education-spending",
        "superfund-right-to-sue", "crime", "duty-free-exports",
        "export-administration-act-south-africa"]

df = pd.read_csv(url, header=None, names=cols, na_values='?')
print(df.shape)
df.head()

(435, 17)


,Class Name,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [0]:
df = df.replace({'y':1, 'n':0})

In [0]:
reps = df[df['Class Name'] == 'republican']
dems = df[df['Class Name'] == 'democrat']

In [4]:
print(reps.head())
reps.shape

    Class Name  ...  export-administration-act-south-africa
0   republican  ...                                     1.0
1   republican  ...                                     NaN
7   republican  ...                                     1.0
8   republican  ...                                     1.0
10  republican  ...                                     0.0

[5 rows x 17 columns]


(168, 17)

In [5]:
print(dems.head())
dems.shape

  Class Name  ...  export-administration-act-south-africa
2   democrat  ...                                     0.0
3   democrat  ...                                     1.0
4   democrat  ...                                     1.0
5   democrat  ...                                     1.0
6   democrat  ...                                     1.0

[5 rows x 17 columns]


(267, 17)

## Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01

In [0]:
from scipy.stats import ttest_ind as tind

issues = ["handicapped-infants", "water-project-cost-sharing",
        "adoption-of-the-budget-resolution", "physician-fee-freeze", 
        "el-salvador-aid", "religious-groups-in-schools",
        "anti-satellite-test-ban", "aid-to-nicaraguan-contras", "mx-missile",
        "immigration", "synfuels-corporation-cutback", "education-spending",
        "superfund-right-to-sue", "crime", "duty-free-exports",
        "export-administration-act-south-africa"]

dem_supported = []
for issue in issues:
    # if there are more dem yes votes than rep yes votes
    if ((dems[issue].sum() > reps[issue].sum()) & (
        # and it's by a strong majority
        tind(dems[issue], reps[issue], nan_policy='omit').pvalue < 0.01)):
        
        #add it to a list
        dem_supported.append(issue)

Democrats support the following issues more than republicans do:

In [7]:
dem_supported

['handicapped-infants',
 'adoption-of-the-budget-resolution',
 'anti-satellite-test-ban',
 'aid-to-nicaraguan-contras',
 'mx-missile',
 'synfuels-corporation-cutback',
 'duty-free-exports',
 'export-administration-act-south-africa']

## Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01

In [0]:
# Exactly the same except '>' is now '<' and the list
# and print statements are republican.
rep_supported = []

for issue in issues:
    # if there are more rep yes votes than dem yes votes
    if (dems[issue].sum() < reps[issue].sum()) & (
        # and it was by a strong majority
        tind(dems[issue], reps[issue], nan_policy='omit').pvalue < 0.01):
        
        # add it to a list.
        rep_supported.append(issue)

Republicans support the following more than Democrats do:

In [9]:
rep_supported

['physician-fee-freeze',
 'el-salvador-aid',
 'religious-groups-in-schools',
 'education-spending',
 'superfund-right-to-sue',
 'crime']

## Issues where Democrats and Republicans are tied.

In [10]:
# Using hypothesis testing, find an issue where the difference between
# republicans and democrats has p > 0.1
# (i.e. there may not be much of a difference)
for issue in issues:
    if (tind(reps[issue], dems[issue], nan_policy='omit').pvalue > 0.01):
        print(f'Republicans and Democrats are tied on {issue}.')

Republicans and Democrats are tied on water-project-cost-sharing.
Republicans and Democrats are tied on immigration.


## As a function:

In [0]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind as tind

def support(df, class_column, class_labels, attribute_columns, alpha):
    """ 
    Performs a series of 2-tailed t-tests between two classes
    along a list of attributes. Omits nan values.

    - df: The dataframe containing the data for testing

    - class_column: The column label in the dataframe where the class labels are located

    - class_labels: A list of two class labels to compare

    - attribute_columns: A list of labels containing columns to test the classes on

    - alpha: The level of significance desired for ttest results.
    """
    # Create subsets of the dataframe for each class
    subsets = []
    for label in class_labels:
        subsets.append(df[df[class_column]==label])

    # Lists to store the attribute labels when we're done
    null_affinities = []
    affinities = [null_affinities]
    for x in class_labels:
        affinities.append([])

    # Start testing
    for attribute_label in attribute_columns:
            if (tind(subsets[0][attribute_label], subsets[1][attribute_label], nan_policy='omit').pvalue > alpha):
                affinities[0].append(attribute_label)
            elif (subsets[0][attribute_label].sum() > subsets[1][attribute_label].sum()):
                affinities[1].append(attribute_label)
            else:
                affinities[2].append(attribute_label)

    # return a list of lists showing which classes the attributes are
    # affined with. ([0] for nulls, [1] for class_labels[0], etc.)
    return affinities

    #TODO: extend to an arbitrary number of classes and dataframes

In [0]:
ttests = support(df=df, class_column='Class Name', class_labels=['democrat', 'republican'], attribute_columns=df.columns[1:], alpha=0.01)

In [13]:
ttests

[['water-project-cost-sharing', 'immigration'],
 ['handicapped-infants',
  'adoption-of-the-budget-resolution',
  'anti-satellite-test-ban',
  'aid-to-nicaraguan-contras',
  'mx-missile',
  'synfuels-corporation-cutback',
  'duty-free-exports',
  'export-administration-act-south-africa'],
 ['physician-fee-freeze',
  'el-salvador-aid',
  'religious-groups-in-schools',
  'education-spending',
  'superfund-right-to-sue',
  'crime']]

## No difference between classes along the following attributes

In [14]:
ttests[0]

['water-project-cost-sharing', 'immigration']

## Class 1 ('democrat') more strongly related to the following than Class 2 ('republican'):

In [15]:
ttests[1]

['handicapped-infants',
 'adoption-of-the-budget-resolution',
 'anti-satellite-test-ban',
 'aid-to-nicaraguan-contras',
 'mx-missile',
 'synfuels-corporation-cutback',
 'duty-free-exports',
 'export-administration-act-south-africa']

## Class 2 more strongly related to the following than Class 1:

In [16]:
ttests[2]

['physician-fee-freeze',
 'el-salvador-aid',
 'religious-groups-in-schools',
 'education-spending',
 'superfund-right-to-sue',
 'crime']